# Dataset with extracted dialogues

In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('/content/drive/MyDrive/masters/try/last dataset/edited_dataset.csv')

# Import libraries 

In [ ]:
pip install natasha

In [ ]:
pip install pymorphy2

In [ ]:
from nltk.tokenize import word_tokenize 
from nltk import download as nltk_download 
nltk_download ('punkt')

In [6]:
from natasha import (Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsSyntaxParser, NewsNERTagger, Doc)
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

In [7]:
from pymorphy2 import MorphAnalyzer
parser = MorphAnalyzer()

In [ ]:
!pip install gensim

In [9]:
import gensim

# Initial black list

In [ ]:
!wget https://raw.githubusercontent.com/dhhse/dh2020/master/data/stop_ru.txt
with open ('stop_ru.txt', 'r') as stop_file:
    rus_stops = [word.strip() for word in stop_file.readlines()]

In [11]:
punctuation = '!\"#$%&\'()*+,-./:;<=>?@[\]^_`{|}~—»«...–'


# Topic Modeling

In [12]:
import re

In [13]:
def preprocess(input_text):
    tokenized_text = word_tokenize(input_text.lower())
    output_text = [word for word in tokenized_text if word not in filters]
    clean_text = [word for word in output_text if word.isalpha()]
    lemmatized_text = [parser.parse(word)[0].normal_form for word in clean_text]
    return lemmatized_text

In [14]:
authors = []
titles = []
topics = []

In [ ]:
for index in range(len(dataset['dialogues'].tolist())):
  text = dataset['dialogues'].tolist()[index]
  text_list = []
  text_list.append(text)
  doc_natasha = Doc(text)
  doc_natasha.segment(segmenter)
  doc_natasha.tag_morph(morph_tagger)
  doc_natasha.parse_syntax(syntax_parser)
  doc_natasha.tag_ner(ner_tagger)

  names = []
  for span in doc_natasha.spans:
    if span.type == 'PER':
      if span.text not in names:
        names.append(span.text)
      span.normalize(morph_vocab)
      if span.normal not in names:
        names.append(span.normal)
  filters = rus_stops + list(punctuation)
  for name in names:
    word = name.lower()
    filters = filters + word.split(' ')
  preprocessed_texts = [preprocess(text_t) for text_t in text_list]
  gensim_dictionary_for_TM = gensim.corpora.Dictionary(preprocessed_texts)
  gensim_dictionary_for_TM.compactify()
  corpus = [gensim_dictionary_for_TM.doc2bow(text_p) for text_p in preprocessed_texts]
  lda = gensim.models.LdaMulticore(corpus,num_topics = 10, id2word=gensim_dictionary_for_TM, passes=10)
  topics_book = []
  for topic in lda.print_topics():
    topic_extracted = re.findall(r'(?<=")\w.*?(?=")', topic[1])
    for i in topic_extracted:
      if i not in topics_book:
        topics_book.append(i)
  
  author = dataset['author'].tolist()[index]
  authors.append(author)
  # extract book title 
  title = dataset['book'].tolist()[index]
  titles.append(title)
  # topics 
  topics.append(topics_book)

df_columns = ['author', 'book', 'topics']
dataframe = pd.DataFrame(columns = df_columns)
dataframe['author'] = authors
dataframe['book'] = titles
dataframe['topics'] = topics

# creating a dataset 
dataframe.to_csv('topics_dialogues.csv')

In [ ]:
!pip install pyldavis

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

In [ ]:
vis = gensimvis.prepare(lda, corpus, gensim_dictionary_for_TM)

In [ ]:
pyLDAvis.enable_notebook()

In [ ]:
vis